#  What properties of a population of neurons contributes to their effectiveness in decoding eye movement behavior? 

## Directional Tuning:



In [1]:
import pickle
import numpy as np
from load_pickle import get_outputs
from itertools import product
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import io
from scipy import stats
import pandas as pd
import seaborn as sns
import os
import sys

cwd = os.getcwd()
sys.path.append(cwd+"/../handy_functions")

from scipy.stats import mannwhitneyu, normaltest, circvar
from statannotations.Annotator import Annotator
from scipy.stats import circvar
from utils import *
from pull_pickles import make_DF, print_df

data_folder = '/Users/kendranoneman/Projects/mayo/data/neural-decoding/outpickles/'
fig_folder='/Users/kendranoneman/Projects/mayo/NeuralDecoding/figures/neuralPop_accuracy/' 
colors = ['mediumorchid','seagreen','cornflowerblue','gold']
sessions = [18,20,22,24,28,29,32,33,34,38]

units = pd.DataFrame()
for s in sessions:
    u = pd.read_csv(cwd+'/../datasets/units-pa{:0>2d}dir4A-pre500-post300.csv'.format(s))
    units = pd.concat([units,u],ignore_index=True)

#make_DF(sessions)
DF = pd.read_csv(cwd+'/out.csv')
DF.head(3)

,sess,repeat,outerFold,nMT,nFEF,R2,rho,neuron_inds,snr_mn,snr_sem,...,vfr_sem,dm_mn,dm_sem,di_mn,di_sem,si_mn,si_sem,R2_norm,condition,num_neurons
0,18,0,0,0,2,0.001751,0.043729,[ 8 19],1.060119,0.002384,...,2.821905,0.023296,0.004286,0.038874,0.011970,0.012080,0.001887,0.766573,fef only,2
1,18,1,0,0,2,0.001152,0.040728,[11 21],1.054688,0.046538,...,0.490034,0.010924,0.005565,0.021491,0.010891,0.006942,0.002211,0.766060,fef only,2
2,18,2,0,0,2,0.002906,0.112175,[ 9 21],0.962384,0.018730,...,5.002747,0.014199,0.007881,0.027763,0.015326,0.008086,0.003020,0.767563,fef only,2


In [7]:
df_selec = DF.groupby(['nMT','nFEF','num_neurons','sess','condition','repeat','outerFold'])[['R2','rho','snr_mn','snr_sem']].mean().reset_index()
df_selec.head(3)

,nMT,nFEF,num_neurons,sess,condition,repeat,outerFold,R2,rho,snr_mn,snr_sem
0,0,2,2,18,fef only,0,0,0.001751,0.043729,1.060119,0.002384
1,0,2,2,18,fef only,0,1,0.002247,0.049726,1.060119,0.002384
2,0,2,2,18,fef only,0,2,0.002473,0.046773,1.060119,0.002384


In [8]:
# Define function to output plot of the model coefficients
def coefplot(results,axes,col,reorderlist):
    coef_df = pd.DataFrame(results.summary().tables[1].data)
    coef_df.columns = coef_df.iloc[0]
    coef_df=coef_df.drop(0)
    coef_df = coef_df.set_index(coef_df.columns[0])
    coef_df = coef_df.astype(float)
    
    errors = coef_df['coef'] - coef_df['[0.025']
    coef_df['errors'] = errors
    coef_df = coef_df.drop(['const'])
    coef_df = coef_df.sort_values(by=['coef'])

    variables = list(coef_df.index.values)
    coef_df['variables'] = variables
    #coef_df.sort_index(inplace=True,ascending=False) 
    coef_df = coef_df.iloc[pd.Categorical(coef_df.index,reorderlist).argsort()]
    
    if axes!=0:
        a = sns.pointplot(ax=axes, data=coef_df, x="variables", y="coef",errorbar="se", capsize=.2, color=col, order=reorderlist)
        coef_df.plot(x='variables', y='coef', kind='bar', 
                     ax=axes, color='none',error_kw=dict(ecolor=col, lw=2, capsize=5, capthick=2), 
                     yerr='errors', legend=False)
        axes.axhline(y=0, linestyle='--', color='black', linewidth=1)
    else:
        ax = plt.gca()
        a = sns.pointplot(ax=ax, data=coef_df, x="variables", y="coef",errorbar="se", capsize=.4, color=col, order=reorderlist)
        coef_df.plot(x='variables', y='coef', kind='bar', 
                     ax=ax, color='none',error_kw=dict(ecolor=col, lw=2, capsize=5, capthick=2), 
                     yerr='errors', legend=False)
        ax.axhline(y=0, linestyle='--', color='black', linewidth=1)
    
    a.set(xlabel=None)
    a.set(ylabel=None)
    
    return a

## Preferred direction

In [ ]:
n_neurons = 12
df = df_selec.loc[(df_selec['num_neurons'] == n_neurons) & (df_selec['condition'] != 'mt+fef')]
palette = {'mt only': colors[0],'fef only': colors[1],'mt=fef': colors[2]}
hue_order = [ 'mt only','fef only', 'mt=fef']

a = sns.lmplot(data=df, x="snr_mn", y="R2", hue="condition", col="sess", col_wrap=5, palette=palette, hue_order=hue_order, scatter_kws={"s": 3,'alpha':0.5}, height=3,
    line_kws={"lw":3}, truncate=False,facet_kws=dict(sharex=False, sharey=False))
def specs(data,**kws):
    ax = plt.gca()

    g = data.condition.unique()[0]
    if g=='mt only':
        col = colors[0]
        y_pos = 0.2
        
    elif g=='fef only':
        col = colors[1]
        y_pos = 0.15
    else:
        col = colors[2]
        y_pos = 0.1

    r, p = stats.pearsonr(data['snr_mn'], data['R2'])
    ax.text(0.8, y_pos, r"$\rho$"+'={:.2f} ({:.3f})'.format(r, p),color=col, transform = ax.transAxes)

_ = a.map_dataframe(specs)
a.tight_layout(w_pad=1)
a.set(xlabel=None)
a.set(ylabel=None)

a.fig.text(0.47, 0, 'Mean SNR', ha='center', fontsize=14)
a.fig.text(0.001, 0.5, 'Prediction Accuracy ($R^2$)', va='center', rotation='vertical', fontsize=14)

a.fig.text(0.47, 0.98, 'Correlations between SNR and $R^2$ ({} total neurons)'.format(n_neurons), ha='center', fontsize=16)
a.savefig(fig_folder+'mnsnrVr2_corrPerSess.png', dpi=300, bbox_inches='tight')  

In [ ]:
############ varPrefDir v. R2: individual sessions ################

from scipy import stats

df = DF.groupby(['num_neurons','sess','condition','repeat'])[['R2','rho','snr_mn','snr_sem','pd_sem','mfr_mn','mfr_sem','vfr_mn','vfr_sem','dm_mn','dm_sem','di_mn','di_sem','si_mn','si_sem']].mean().reset_index()
#df = DF
df = df.loc[(df['num_neurons'] == 12) & (df['condition'] != 'mt+fef')]
#df['R2_zscore'] = df.groupby(['condition','sess']).R2.transform(lambda x : stats.zscore(x,ddof=1)).values
#df['snr_mn_zscore'] = df.groupby(['condition','sess']).snr_mn.transform(lambda x : stats.zscore(x,ddof=1)).values
df.sort_values(by='condition', ascending=False)
palette = {'mt only': colors[0],'fef only': colors[1],'mt=fef': colors[2]}
hue_order = [ 'fef only', 'mt only','mt=fef']

a = sns.relplot(data=df, x="pd_sem", y="R2", hue="condition",hue_order=hue_order,palette=palette,col='sess',col_wrap=5,size=df['num_neurons'],facet_kws={'sharey': False, 'sharex': True})
def specs(x,y,hue,**kwargs):
    ax = plt.gca()
    p = stats.pearsonr(x[hue=='mt only'].values, y[hue=='mt only'].values)
    ax.text(0.8, 0.9, r"$\rho$"+'={:.2f} ({:.3f})'.format(p.statistic, p.pvalue),color=colors[0], transform = ax.transAxes)
    p = stats.pearsonr(x[hue=='fef only'].values, y[hue=='fef only'].values)
    ax.text(0.8, 0.85, r"$\rho$"+'={:.2f} ({:.3f})'.format(p.statistic, p.pvalue),color=colors[1], transform = ax.transAxes)
    p = stats.pearsonr(x[hue=='mt=fef'].values, y[hue=='mt=fef'].values)
    ax.text(0.8, 0.8, r"$\rho$"+'={:.2f} ({:.3f})'.format(p.statistic, p.pvalue),color=colors[2], transform = ax.transAxes)
    ax.set_xlim((0,70))
a.map(specs,'pd_sem','R2','condition')

a.tight_layout(w_pad=1)
a.set(xlabel=None)
a.set(ylabel=None)


a.fig.text(0.47, 0.01, 'Preferred Direction (circular SEM)', ha='center', fontsize=16)
a.fig.text(0.001, 0.5, '$R^2$', va='center', rotation='vertical', fontsize=16)

a.savefig(fig_folder+'pdvarVr2.png', dpi=300, bbox_inches='tight')

In [ ]:
############ varPrefDir v. R2: all sessions ################
from scipy import stats

df = DF.groupby(['num_neurons','sess','condition','repeat'])[['R2','rho','snr_mn','snr_sem','pd_sem','mfr_mn','mfr_sem','vfr_mn','vfr_sem','dm_mn','dm_sem','di_mn','di_sem','si_mn','si_sem']].mean().reset_index()
#df = DF
df = df.loc[(df['num_neurons'] == 12) & (df['condition'] != 'mt+fef')]
#df['R2_zscore'] = df.groupby(['condition','sess']).R2.transform(lambda x : stats.zscore(x,ddof=1)).values
#df['snr_mn_zscore'] = df.groupby(['condition','sess']).snr_mn.transform(lambda x : stats.zscore(x,ddof=1)).values
df.sort_values(by='condition', ascending=False)
palette = {'mt only': colors[0],'fef only': colors[1],'mt=fef': colors[2]}
hue_order = ['fef only', 'mt only','mt=fef']

a = sns.relplot(data=df, x="pd_sem", y="R2", hue="condition",palette=palette,size=df['num_neurons'])
def specs(x,y,hue,**kwargs):
    ax = plt.gca()
    p = stats.pearsonr(x[hue=='mt only'].values, y[hue=='mt only'].values)
    ax.text(0.8, 0.9, r"$\rho$"+'={:.2f} ({:.3f})'.format(p.statistic, p.pvalue),color=colors[0], transform = ax.transAxes)
    p = stats.pearsonr(x[hue=='fef only'].values, y[hue=='fef only'].values)
    ax.text(0.8, 0.85, r"$\rho$"+'={:.2f} ({:.3f})'.format(p.statistic, p.pvalue),color=colors[1], transform = ax.transAxes)
    p = stats.pearsonr(x[hue=='mt=fef'].values, y[hue=='mt=fef'].values)
    ax.text(0.8, 0.8, r"$\rho$"+'={:.2f} ({:.3f})'.format(p.statistic, p.pvalue),color=colors[2], transform = ax.transAxes)
    ax.set_xlim((0,70))

a.map(specs,'pd_sem','R2','condition')
a.tight_layout(w_pad=1)
a.set(xlabel=None)
a.set(ylabel=None)

a.fig.text(0.47, 0.01, 'Preferred Direction (circular SEM)', ha='center', fontsize=14)
a.fig.text(0.001, 0.5, 'Prediction Accuracy ($R^2$)', va='center', rotation='vertical', fontsize=14)

a.savefig(fig_folder+'pdvarVr2_allSess.png', dpi=300, bbox_inches='tight')

## Based on the mean SNR and sem PD of a cluster of neurons, what was the decoding accuracy?

In [ ]:
fig, axes = plt.subplots(1,3,figsize=(16,4)) #Create figure (11 rows by 3 columns)
fig.tight_layout(pad=1.5)
fig.subplots_adjust(left=0.01)

nn = 12
n_bins = 8

df = DF.loc[DF['condition'] != 'mt+fef'] #.loc[DF['sess'] == sessions[s]]
df = df.loc[(df['num_neurons'] == nn)]
#df = df.loc[(df['num_neurons'] <= np.array([df['nMT'].max(), df['nFEF'].max()]).min()) & (df['condition'] != 'mt+fef')]
#df = df.groupby(['num_neurons','sess','condition','repeat'])[['R2','rho','snr_mn','snr_sem','pd_sem','mfr_mn','mfr_sem','vfr_mn','vfr_sem','dm_mn','dm_sem','di_mn','di_sem','si_mn','si_sem']].mean().reset_index()

df['snr_binned'] = pd.qcut(df['snr_mn'], n_bins)
df["snr_bin"] = df["snr_binned"].apply(lambda x: round(x.left,1))
df['pd_binned'] = pd.qcut(df['pd_sem'], n_bins)
df["pd_bin"] = df["pd_binned"].apply(lambda x: round(x.left,1))


df_t = df.groupby(['snr_bin', 'pd_bin','condition'])["R2"].mean().reset_index(name="mean_R2")

# MT only
df2 = df.loc[(df['condition'] == 'mt only')]
df2 = df2[['snr_bin','pd_bin','R2']]
df_m = df2.groupby(['snr_bin', 'pd_bin'])["R2"].mean().reset_index(name="mean_R2")
result = df_m.pivot(index='snr_bin', columns='pd_bin', values='mean_R2')

a = sns.heatmap(ax=axes[0],data=result, cmap=sns.light_palette("mediumorchid", as_cmap=True), mask=result.isnull(), vmin=df_t['mean_R2'].min(), vmax=df_t['mean_R2'].max())
a.invert_yaxis()
a.set(xlabel=None)
a.set(ylabel=None)
a.set_title('{} MT neurons'.format(nn),loc='center')

# FEF only
df2 = df.loc[(df['condition'] == 'fef only')]
df2 = df2[['snr_bin','pd_bin','R2']]
df_m = df2.groupby(['snr_bin', 'pd_bin'])["R2"].mean().reset_index(name="mean_R2")
result = df_m.pivot(index='snr_bin', columns='pd_bin', values='mean_R2')

a = sns.heatmap(ax=axes[1],data=result, cmap=sns.light_palette("seagreen", as_cmap=True), mask=result.isnull(), vmin=df_t['mean_R2'].min(), vmax=df_t['mean_R2'].max())
a.invert_yaxis()
a.set(xlabel=None)
a.set(ylabel=None)
a.set_title('{} FEF neurons'.format(nn),loc='center')
a.set(yticklabels=[])

# FEF only
df2 = df.loc[(df['condition'] == 'mt=fef')]
df2 = df2[['snr_bin','pd_bin','R2']]
df_m = df2.groupby(['snr_bin', 'pd_bin'])["R2"].mean().reset_index(name="mean_R2")
result = df_m.pivot(index='snr_bin', columns='pd_bin', values='mean_R2')

a = sns.heatmap(ax=axes[2],data=result, cmap=sns.light_palette("cornflowerblue", as_cmap=True), mask=result.isnull(), vmin=df_t['mean_R2'].min(), vmax=df_t['mean_R2'].max())
a.invert_yaxis()
a.set(xlabel=None)
a.set(ylabel=None)
a.set_title('{} MT, {} FEF neurons'.format(int(nn/2),int(nn/2)),loc='center')
a.set(yticklabels=[])

fig.text(0.99, 0.5, 'Prediction Accuracy ($R^2$)', va='center', rotation=-90, fontsize=12)
fig.text(0, 0.5, 'SNR (mean)', va='center', rotation='vertical', fontsize=14)
fig.text(0.5, 0, 'Preferred Direction (circular SEM)', ha='center', fontsize=14)